## Import Libraries

In [9]:
# %matplotlib ipympl
# %matplotlib inline
%matplotlib wx

In [10]:
import matplotlib.pyplot as plt
plt.ion()

In [11]:
from pydgilib_extra import *
from atprogram.atprogram import atprogram

In [12]:
from os import getcwd, path, pardir

## Compile and program project

In [13]:
project_path = path.join(getcwd(), "FFT-S")
project_path

'C:\\Users\\erikw_000\\Documents\\GitHub\\Atmel-SAML11\\Experiments\\Workloads\\FFT\\FFT-S'

In [46]:
atprogram(project_path, verbose=2)

make: Nothing to be done for 'all'.

[DEBUG] Starting execution of "chiperase"
[DEBUG] Starting process 'C:\Program Files (x86)\Atmel\Studio\7.0\atbackend\atbackend.exe'
[DEBUG] Connecting to TCP:127.0.0.1:32082
[INFO] Connected to edbg, fw version: 3.25
[INFO] Firmware check OK
[DEBUG] Command "chiperase" finished with return code 0
[DEBUG] Starting execution of "program"
[DEBUG] Memory segment base written at 0x00000000. Size = 0x00005800.
[DEBUG] Memory segment base written at 0x00005800. Size = 0x00000068.
[DEBUG] Memory segment base written at 0x0000fb00. Size = 0x00000020.
[DEBUG] Memory segment base written at 0x00804000. Size = 0x00000020.
[DEBUG] Command "program" finished with return code 0
[DEBUG] Exit successfully.
Firmware check OK
Chiperase completed successfully
Programming completed successfully.



0

## Data Logging

In [15]:
live_plot = True

Create a figure for the plot.

In [16]:
if live_plot:
    fig = plt.figure(figsize=(10, 6))
    fig.show()

Create the configuration dictionary for `DGILibExtra`.

In [57]:
if live_plot:
    config_dict = {
        "loggers": [LOGGER_OBJECT, LOGGER_PLOT, LOGGER_CSV],
        "plot_pins": [False, False, True, True],
        "gpio_delay_time" : 0.0007,
        "plot_pins_method": "line",
        "plot_xmax": 6,
        "fig": fig,
        "window_title": "Experiment FFT",
        "file_name_base": "experiment_fft"
    }
else:
    config_dict = {
        "loggers": [LOGGER_OBJECT, LOGGER_CSV],
        "gpio_delay_time" : 0.0007,
        "file_name_base": "experiment_fft"
    }

Stop criteria to pass to the logger:

In [58]:
def stop_fn(logger_data):
    return all(logger_data.gpio.values[-1])

Stop criteria to pass to the parser:

In [59]:
def stop_function(pin_values):
    return all(pin_values)

In [60]:
repetitions = 10

Perform the measurement.

In [61]:
if live_plot:
    fig.clf()
    for ax in fig.get_axes():
        ax.cla()
        
charges_allocate = []
charges_fft = []
charges_switch_to_ifft = []
charges_ifft = []
charges_free = []
times_allocate = []
times_fft = []
times_switch_to_ifft = []
times_ifft = []
times_free = []


with DGILibExtra(**config_dict) as dgilib:
    for _ in range(repetitions):
        dgilib.device_reset()
        dgilib.logger.log(10,stop_fn)
        charge2, time2 = power_and_time_per_pulse(dgilib.data, 2, 0, stop_function=stop_function)
        charge3, time3 = power_and_time_per_pulse(dgilib.data, 2, 0, stop_function=stop_function)
        
        if len(charge2) != 3 or len(time2) != 3 or len(charge3) != 3 or len(time3) != 3:
            print(f"Parsing of measurement data failed, charge2:{charge2}, time2:{time2}, charge3:{charge3}, time3:{time3},")
            break
        charges_allocate.append(charge2[0])
        charges_fft.append(charge3[0])
        charges_switch_to_ifft.append(charge2[1])
        charges_ifft.append(charge3[1])
        charges_free.append(charge2[2])
        times_allocate.append(time2[0])
        times_fft.append(time3[0])
        times_switch_to_ifft.append(time2[1])
        times_ifft.append(time3[1])
        times_free.append(time2[2])
        # dgilib.logger.plotobj.ax.set_title(f"Previous sample: charge: {sum(charge2+charge3)*1e3:.06} mC, time: {sum(time2+time3):.06} s")
        dgilib.empty_data()

    charges = charges_allocate + charges_fft + charges_switch_to_ifft + charges_ifft + charges_free 
    times = times_allocate + times_fft + times_switch_to_ifft + times_ifft + times_free

    dgilib.logger.plotobj.ax.set_title(f"Average of {repetitions} samples: charge: {sum(charges)/len(charges)*1e3:.06} mC, time: {sum(times)/len(times):.06} s")

Parsing of measurement data failed, charge2:[1.6025214620356904e-05], time2:[0.04864593066666667], charge3:[1.6025214620356904e-05], time3:[0.04864593066666667],


ZeroDivisionError: division by zero

In [40]:
import pandas as pd

In [52]:
df = pd.DataFrame({'times': times, 'charges':charges})

In [53]:
df

,times,charges
0,0.503246,2.479505e-04
1,0.503402,2.481504e-04
2,0.504051,2.483048e-04
3,0.502662,2.475527e-04
4,0.503072,2.477944e-04
5,0.503912,2.480753e-04
6,0.503926,2.480331e-04
7,0.503860,2.481267e-04
8,0.504581,2.485862e-04
9,0.504324,2.483452e-04


In [54]:
df.describe()

,times,charges
count,50.000000,5.000000e+01
mean,0.403133,1.982598e-04
std,0.203580,1.001250e-04
min,0.000066,2.110338e-08
25%,0.503072,2.474800e-04
50%,0.504051,2.475823e-04
75%,0.504111,2.480331e-04
max,0.504581,2.485862e-04


In [55]:
sum(charges)/repetitions

0.0009912991028630635

In [56]:
print("%.0100f" % (sum(charges)/repetitions))

0.0009912991028630635092361300664265399973373860120773315429687500000000000000000000000000000000000000
